如果要加入新資料
1. NewDataLoader(BasicLoader)
2. LoaderMapping新增欄位
3. .yml新增欄位

In [1]:
### reload magic
%reload_ext autoreload
%autoreload 2

import sys
import yaml
import torch
import torchvision
import torch.utils.cpp_extension

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from src.data_manager import DataManager
from src.model_director import ModelDirector

# os.environ['CUDA_VISIBLE_DEVICES']="0"
print('Python version', sys.version)
print('CUDA_HOME', torch.utils.cpp_extension.CUDA_HOME)
print('CudaToolKit Version', torch.version.cuda)
print('Cudnn Version', torch.backends.cudnn.version())
print('torch Version', torch.__version__)
print('torchvision Version', torchvision.__version__)

config_file = './exp_config/exp1.yml'
with open(config_file, "r") as content:
    config = yaml.safe_load(content) 

Python version 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
CUDA_HOME /usr/local/cuda
CudaToolKit Version 11.8
Cudnn Version 8700
torch Version 2.0.0+cu118
torchvision Version 0.15.1+cu118


In [2]:
dm = DataManager(
    data_meta_info=config['train_config']['data_meta_info'], 
    **config['train_config']['data_loader_params']
)

100%|██████████| 362024/362024 [00:02<00:00, 158253.55it/s]


RainLoaderJay  instantiated.


100%|██████████| 366871/366871 [00:02<00:00, 148398.61it/s]


RadarLoaderJay  instantiated.
[DataManager] Training Data Size: 30978; Validating Data Size: 10326; Testing Data Size: 10326


In [6]:
model = ModelDirector(
    data_info = dm.get_data_info(),
    **config['model'],
).build_model()

[EncoderParams] channel_count:2 Shape:(120, 120)
[EncoderParams_Poni] channel_count: 1
[ForecasterPONI] TargetLen:3 TeacherForcing:0.5
Using CPN_PONI model
[Discriminator] Downsample:3
[GANFramework Disc_Weight:0.01] Ckp:/wk171/handsomedong/deepQPF_research/checkpoint/


In [ ]:
logger = TensorBoardLogger(save_dir='logs', name=config['model']['name'])
checkpoint_callback = model.get_checkpoint_callback()
trainer = Trainer(
    benchmark=True,
    accelerator='gpu',
    devices=[0],
    max_epochs=50, 
    fast_dev_run=False, 
    logger=logger,
    check_val_every_n_epoch=1,
    callbacks=[checkpoint_callback, EarlyStopping(monitor="val_loss", patience=5)],
)
trainer.fit(model, dm)